In [7]:
%load_ext autoreload
%autoreload 2

In [11]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import minimize
from typing import Tuple, Optional, Dict

df = pd.read_excel('Data/P2_processed_data.xlsx')
print (df.head())

    波长      Blue  Green  Red  Warm White  Cold White
0  380  0.000000    0.0  0.0         0.0    0.021609
1  381  0.001835    0.0  0.0         0.0    0.016859
2  382  0.005505    0.0  0.0         0.0    0.011873
3  383  0.002752    0.0  0.0         0.0    0.027545
4  384  0.000000    0.0  0.0         0.0    0.029682


In [24]:
def linear_combine_spd(spd_df: pd.DataFrame, w: np.ndarray) -> pd.DataFrame:
   
    # 确保顺序
    channels = ['Blue', 'Green', 'Red', 'Warm White', 'Cold White']
    assert len(w) == len(channels)
    
    # 取出各通道光谱矩阵 (波长数 × 通道数)
    channel_power = spd_df[channels].values 
    
    # 线性叠加：波长点数 × 通道数 乘以 通道数 × 1，结果是 (波长点数,)
    combined_power = np.dot(channel_power, w)

    #return
    result_df = pd.DataFrame({
        'wavelength': spd_df['波长'],
        'power': combined_power
    })
    
    return result_df

In [20]:
#weight init
w = np.array((1,1,1,1,1))
print (w)

#Configeration
CCT_BOUNDS = (5500.0, 6500.0)   # [min, max] K
RG_BOUNDS = (95.0, 105.0)       # [min, max]
WEIGHT_BOUNDS = (0.0, 1.0)      # each weight in [0,1]
NUM_RANDOM_STARTS = 30          # multi-start次数
FEASIBILITY_TOL = 1e-6          # 约束容忍度（数值判断）
MAXITER = 300

[1 1 1 1 1]


In [ ]:
class MetricCache:
    def __init__(self, spd):
        self.spd = spd
        self.cache = {}

    def key_of(self, w: np.ndarray, digits: int = 8):
        # 把权重归一化为 tuple，便于哈希
        return tuple(np.round(w, digits).tolist())
        
    def compute(self, w: np.ndarray):
        k = self.key_of(w)
        
        if k in self.cache:
            return self.cache[k]
        spd = np.dot(w, self.S)  # 合成 SPD
        
        try:
            cct = float(get_CCT(spd))
        except Exception as e:
            cct = float('nan')
        try:
            duv = float(get_duv(spd))
        except Exception:
            duv = float('nan')
        try:
            rf = float(get_rf(spd))
        except Exception:
            rf = float('nan')
        try:
            rg = float(get_rg(spd))
        except Exception:
            rg = float('nan')
        try:
            mel = float(get_mel_DER(spd))
        except Exception:
            mel = float('nan')
        val = {'spd': spd, 'cct': cct, 'duv': duv, 'rf': rf, 'rg': rg, 'mel': mel}
        self.cache[k] = val
        return val